# Evolutionary rescue of positive interactions - Theoretical analysis

This notebook contains the theoretical analysis of evolutionary rescue probability in populations engaged with positive interactions. 

Briefly, Since rescue probability is dependent on  the rescue time window - the time window during which adapted mutants can rise and prevent the population’s extinction - we construct an approximation of this time window first. We approximate the rescue time window by calculating the difference between the time it takes the ancestral population to decline to the critical population size, and the time it takes adapted mutants to grow sufficiently in order to rescue the population from collapse. The approximation of the rescue probability of populations engaged in intrasepceis cooperation and mutualism is then calculated seperately.

Full description of the analysis can be found in **Section 3** in the supplementary information:

https://www.biorxiv.org/content/10.1101/2020.08.06.239608v1


In [25]:
# imports

from matplotlib import pyplot as plt
import numpy as np
from scipy.integrate import quad
import scipy.stats as sts
from scipy.linalg import solve as solve
from scipy import optimize

## Analytic approximations

Time expression:
$$r' = r- \delta$$

$$K' = \frac{Kr'}{r}$$

$$t(N_0,N,r) = \frac{1}{r'}\ln\big(\frac{\frac{K'}{N_0}-1}{\frac{K'}{N}-1}\big)$$

In [30]:
def calc_time(N0,N,r,d,K):  
    ''' 
    Calculate the time it takes species to get from N0 to N 
        N0 - Initial population size
        N - Final population size
        r - Growth rate
        d - Death rate
        K - Carrying cpacity
    return time
    '''
    
    r_new = r - d
    K_new = (K*r_new)/r
    
    return (1/(r_new))*(np.log((K_new/N0-1)/(K_new/N-1)))

Population density expression:

$$N(t) =  \frac{K'N0}{N_0+(K'-N_0)e^{-r't}}$$

In [31]:
def calc_N(t,N0,r,d,K):
    ''' 
    Calculate populations size at time t
        t - timepoint
        N0 - Initial population size
        r - Growth rate
        d - Death rate
        K - Carrying cpacity
    return population size
    '''
    r_new = r - d
    K_new = (K*r_new)/r
    return (K_new*N0)/(N0+(K_new-N0)*np.exp(-r_new*t))

Population density without neglecting competition:

$$N_1(t) =  N_0e^{(r(1-\frac{N_2(t)}{K})-\delta)t}$$

In [48]:
def calc_N_competition(t,N1,N2,r1,r2,d,K):  
    ''' 
    Calculate populations size at time t in the presence of other competing species
        t - timepoint
        N1 - Initial population size 
        N2 - Initial population size of competing species
        r1 - Growth rate
        r2 - Growth rate of competing species
        d - Death rate
        K - Carrying cpacity
    return population size
    '''
    
    return N1*(np.exp((r1*(1-calc_N(t,N2,r2,d,K)/K)-d)*t)) 

Population density integration:
    
$$\int_{0}^{t}N \ dt=   \frac{K'}{r'}\log(\frac{N_0 (e^{r' t}-1)}{K'}+1) $$

In [24]:
def calc_int_N(t,N0,r,d,K):
    ''' 
    Integrate population size in t0=0 to t 
        N0 - Initial population size
        t - Final timepoint
        r - Growth rate
        d - Death rate
        K - Carrying cpacity
    return integrted value
    '''
    if(np.all(t<0)):
        return 0
    r_new = r - d
    K_new = (K*r_new)/r
    return (K_new/r_new)*np.log(np.abs((N0*np.exp(r_new*t)-N0)/K_new+1)) 

Derivative:

$$\frac{dN_i}{dt} = r_i N_i(1-\frac{\sum{N_j}}{K}) - \delta N_i$$


In [26]:
def derivative(N,t,r,K,d):
    '''
    The derivative used in simulations
        N - Population sizes
        t - time 
        r - Growth rates
        K - Carrying capacity
        d - Death rate 
    return derivative
    '''
    dN = r*N*(1-(np.sum(N))/K)- d*N
    return dN

In [42]:
def derivative_total(M,A,ra,rm,K,d):
    ''' 
    Calculate the derivative of total population size (N=A+M)
        A - Ancestor population size
        M - Mutant population size
        r - Ancestor growth rate 
        rm - Mutatnt growth rate
        d - Death rate
        K - Carrying cpacity
    return total populations size
    '''
    r = np.array([ra,rm])
    return derivative(np.array([A,M]),0,r,K,d).sum()

The time at which the mutant reaches steady state:

$$t_c = \frac{K'(r_M'-\delta)}{2r_M'}$$

In [121]:
def calc_tc(rm,d,K):
    ''' 
    Calculate the time mutant reaches steady state
        rm - Growth rate
        d - Death rate
        K - Carrying cpacity
    return the time
    '''
    r_new= rm-d
    K_new = ((K/2)*r_new)/rm
    
    return calc_time(1,K_new-1,rm,d,K)


time-dependent carrying capacity:

$$K(t) = \frac{K'/2}{1+e^{r_M'(t-t_c)}}$$

In [133]:
def K_var(t,r,rm,d,K):
    ''' 
    time-dependent carrying capacity
        r - Ancestor growth rate
        rm - Mutant growth rate
        d - Death rate
        K - Fixed carrying cpacity
    return the carrying capacity
    '''
    r_new = r - d
    K_new = ((K/2)*r_new)/r
    
    tc = calc_tc(rm,d,K)
    return (K_new)/(1+np.exp((t-tc)*(rm-d)))


In [134]:
def calc_N_var_K(t,N0,r,rm,d,K):
    ''' 
    Caculate the population density with time-dependent carrying capacity
        t - timepoint
        r - Ancestor growth rate
        rm - Mutant growth rate
        d - Death rate
        K - Fixed carrying cpacity
    return the carrying capacity
    '''  
    
    r_new = r - d
    K_new = K_var(t,r,rm,d,K)
    
    return (K_new*N0)/(N0+(K_new-N0)*np.exp(-r_new*t))

## Rescue time window

$$\quad t_m = t(A_{0},N_{c},r_A) - t(1,M_{final},r_{M}) $$

In [78]:
def rescue_time_window(r,rm,d,mu,Nc,K):
    ''' 
    Calculate the rescue time window of population engaged in intraspecies cooperation
        r - Ancestor growth rate 
        rm - Mutatnt growth rate
        d - Death rate
        mu - Mutation rate
        Nc - Critical population size
        K - Carrying cpacity
    return rescue time window
    '''
    
    # initial ancestor population size
    A0=(K)*(1-d/rm) 
    
    # Minimal population size of mutants for which total population size is positive
    M_min= optimize.fsolve(derivative_total,K/10,args=(Nc,r,rm1,K,d),full_output=True)[0][0] 
    
    # Final population size of mutants
    M_fin =np.min([Nc,M_min])
    
    
    
    # Numerically extract time for which mutant is above Mfin
    x=np.linspace(0,1000,100000)
    for l in x:
        if calc_N_competition(l,1,A0,rm,r,d,K)>M_fin:
            time1=l
            break    
            
    # Calculate time it takes ancestor to get to Nc
    time2 = calc_time(A0,Nc,r,d,K) 
    
    rescue_time_window = np.max([0,time2- time1])
    
    return rescue_time_window

### Rescue probability of Intraspecies cooperation

In [79]:
def cooperation_Pr(r,rm,d,mu,Nc,K):
    ''' 
    Calculate the rescue probability of population engaged in intraspecies cooperation
        r - Ancestor growth rate 
        rm - Mutatnt growth rate
        d - Death rate
        mu - Mutation rate
        Nc - Critical population size
        K - Carrying cpacity
    return rescue probability
    '''
    
    # initial ancestor population size
    A0=(K)*(1-d/rm) 
    
    # Calculate rescue time window
    rtw = rescue_time_window(r,rm,d,mu,Nc,K)
    
    # Calculate expected number of mutation events
    Mexp = mu*calc_int_N(rtw,A0,r,d,K) 
    
    # Get rescue probability
    rescue_probability = sts.poisson(Mexp).sf(0)
    
    return rescue_probability

## Rescue probability of mutualism

In [136]:
def mutualism_Pr(r,rm,d,mu,Nc,K):
    ''' 
    Calculate the rescue probability of population engaged in mutualism
        r - Ancestor growth rate 
        rm - Mutatnt growth rate
        d - Death rate
        mu - Mutation rate
        Nc - Critical population size
        K - Carrying cpacity
    return rescue probability
    '''
    
    #### First mutational event ####
    
    K_first = K/2
    
    # initial ancestor population size
    A0=(K_first)*(1-d/rm) 
    
    # Calculate rescue time window
    rtw = rescue_time_window(r,rm,d,mu,Nc,K_first)
    
    # Calculate expected number of mutation events
    Mexp1 = mu*calc_int_N(rtw,A0,r,d,K_first)*2 

    
    #### Second mutational event ####
    
    Mexp2 = mu*quad(calc_N_var_K,1,rtw,args=(A0,r,rm,d,K))[0]

    
    # Get rescue probability
    rescue_probability = sts.poisson(Mexp1).sf(0)*sts.poisson(Mexp2).sf(0)
    
    return rescue_probability